### Beam Search

In [8]:
import numpy as np


class MockAutoregressiveModel:
    def __init__(self, vocab_size=5, seed=42):
        self.vocab_size = vocab_size
        self.rng = np.random.default_rng(seed)

    def get_logits(self, input_sequence):
        # Returns random logits for the next token
        # In a real model, this would depend on the input sequence
        return self.rng.normal(size=self.vocab_size)

    def get_softmax(self, input_sequence):
        logits = self.get_logits(input_sequence)
        # Numerical stability: subtract max before exp to prevent overflow
        exp_logits = np.exp(logits - np.max(logits))
        return exp_logits / exp_logits.sum()


def beam_search(model, beam_width=3, max_steps=5, start_token=0, end_token=None):
    # Each beam is a tuple: (sequence, log_prob)
    # Initialize with start token and log probability of 0 (probability 1)
    beams = [([start_token], 0.0)]
    
    for _ in range(max_steps):
        all_candidates = []
        
        # Expand each beam by considering all possible next tokens
        for seq, log_prob in beams:
            # If sequence ended with end_token, don't expand further
            if end_token is not None and seq[-1] == end_token:
                all_candidates.append((seq, log_prob))
                continue
            
            # Get model predictions for next token
            logits = model.get_logits(seq)
            # Convert logits to log probabilities: log(softmax(x)) = x - log_sum_exp(x)
            log_probs = logits - np.log(np.sum(np.exp(logits)))
            
            # Create a candidate for each possible next token
            for idx in range(model.vocab_size):
                candidate_seq = seq + [idx]
                # Add log probability (equivalent to multiplying probabilities)
                candidate_log_prob = log_prob + log_probs[idx]
                all_candidates.append((candidate_seq, candidate_log_prob))
        
        # Keep only the top beam_width sequences by score
        all_candidates.sort(key=lambda x: x[1], reverse=True)  # Sort by log_prob descending
        beams = all_candidates[:beam_width]  # Prune to beam_width
        
    return beams


model = MockAutoregressiveModel(vocab_size=5)
results = beam_search(model, beam_width=3, max_steps=5, start_token=0)
for seq, score in results:
    print("Sequence:", seq, "Score:", score)


Sequence: [0, 3, 1, 2, 0, 1] Score: -4.403832707052807
Sequence: [0, 2, 3, 0, 0, 2] Score: -4.526222266536378
Sequence: [0, 3, 1, 2, 0, 0] Score: -4.746001534201364
